Import all libraries needed

In [89]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Scrap the wikipedia page and remove all not assigned boroughs and assign all not assigned nehiborhoods to the name of its respective borough
Create a dataframe for the toronto downtown area for kmeans cluster

In [299]:

df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)[0]


df = df[df.Borough != "Not assigned"]
toronto_cluster =df[df.Borough == "Downtown Toronto"]



df["Neighbourhood"].replace("Not assigned","Queen's Park", inplace=True)


#for i in df['Neighbourhood']: 
  
  #if i == "Not assigned":
   #  i="Not_assigned"
 



Group all neighbourhoods with the same postal code


In [301]:
comma = lambda a: " , ".join(a) 
#df.groupby(by='Postcode').agg( { 'Borough':comma,'Neighbourhood':comma}).reset_index()
df = df.groupby(['Postcode','Borough'], sort = False).agg(comma)


Calculate the shape of the dataframe

In [302]:

df.shape


(103, 1)

In [304]:
df1=pd.read_csv("http://cocl.us/Geospatial_data")
df1 = df1.rename(columns={'Postal Code': 'Postcode'})


Assign the each postal code with its respective latitude and longitude

In [306]:
df2 = df.join(df1.set_index('Postcode'), on='Postcode')

df2.head()


,,Neighbourhood,Latitude,Longitude
Postcode,Borough,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights , Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494


cluster toronto neighborhoods with a kcluster factor of 5.

In [307]:
#df2 = pd.merge(df, df1, on='Postcode')
#toronto_cluster 
toronto_cluster_merge = pd.merge(toronto_cluster,df1, on='Postcode')
toronto_cluster_merge = toronto_cluster_merge[toronto_cluster_merge.Neighbourhood != "St. James Town"]
kclusters = 5

toronto_cluster_loc = toronto_cluster_merge[["Latitude", "Longitude"]]
#toronto_cluster_loc
#toronto_cluster_loc = toronto_cluster.drop('Neighbourhood', 1)
#toronto_cluster_loc = toronto_cluster_loc[:-2]
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_cluster_loc)


kmeans.labels_.shape


(35,)

In [308]:




columns = ['Neighbourhood']
neighborhoods_venues_sorted = pd.DataFrame(columns=['Neighbourhood'])
#neighborhoods_venues_sorted  
neighborhoods_venues_sorted['Neighbourhood'] = toronto_cluster_merge['Neighbourhood']


neighborhoods_venues_sorted.drop_duplicates(keep=False,inplace=True) 
neighborhoods_venues_sorted.reset_index()
neighborhoods_venues_sorted.shape

(35, 1)

Add the cluster labels to the toronto area dataframe

In [309]:

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_cluster_merge = toronto_cluster_merge.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_cluster_merge # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,4
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,4
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3
8,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568,4
9,M5H,Downtown Toronto,King,43.650571,-79.384568,4
10,M5H,Downtown Toronto,Richmond,43.650571,-79.384568,4


Create a map of the toronto area which shows the clusters around the downtown area.

In [310]:

map = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_cluster_merge['Latitude'], toronto_cluster_merge['Longitude'], toronto_cluster_merge['Neighbourhood'], toronto_cluster_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map)
       
map


